# Notebook for proof reading

**NOTE:** Some part this notebook is from [deeplearning.ai, ChatGPT Prompt Engineering for Developers](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/)

In [1]:
import os

import time

import pandas as pd
import numpy as np
from openai.embeddings_utils import distances_from_embeddings, cosine_similarity
from redlines import Redlines
from IPython.display import display, Markdown, Latex, HTML, JSON


In [2]:
max_tokens = 500
embedding_model = 'text-embedding-ada-002'
model = 'text-davinci-003'

In [3]:
import openai

# setup openai
import os
import openai
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv()
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_CHAT_VERSION") # chatgpt
# openai.api_version = os.getenv("OPENAI_API_VERSION") # davinci
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
def get_completion_chat(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine="chatgpt",
        # model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [5]:
def get_completion_chat_message(messages, model="gpt-3.5-turbo", temperature=0): 

    response = openai.ChatCompletion.create(
        engine="chatgpt",
        # model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [6]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [7]:
def get_completion(prompt, model=model, temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        # stop=stop_sequence,
        model=model,
    )
    # return response.choices[0].message["content"]
    return response["choices"][0]["text"]

## Proof read

Manually add text:

In [8]:
def proof_read_chatgpt(text, debug=False):
    def prepare_prompt(text):
        # prompt = f"proofread and correct the following text: ```{text}```"
        prompt = f"""Proofread and correct the following text
        and rewrite the corrected version. If you don't find
        and errors, just say "No errors found". Don't use 
        any punctuation around the text:
        ```{text}```"""
        return prompt
    
    prompt = prepare_prompt(text)
    if debug:
        print(prompt)
    # response = get_completion(prompt)
    response = get_completion_chat(prompt)
    print(f"Original input:\n{response}")
    print(f"Corrected output:")

    if response == "No errors found.":
        return response
    diff = Redlines(text,response)
    display(Markdown(diff.output_markdown))
    return response



In [9]:
text =f"""
The girl with the black and white puppies have a ball
"""
proof_read_chatgpt(text, debug=False)

Original input:
The girl with the black and white puppies has a ball.
Corrected output:


The girl with the black and white puppies <span style="color:red;font-weight:700;text-decoration:line-through;">have </span><span style="color:red;font-weight:700;">has </span>a <span style="color:red;font-weight:700;text-decoration:line-through;">ball
</span><span style="color:red;font-weight:700;">ball.</span>

'The girl with the black and white puppies has a ball.'